In [2]:
import azureml.core
from azureml.core import Workspace

print(f"Azure ML SDK Version: {azureml.core.VERSION}")

Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.scriptrun = azureml.core.script_run:ScriptRun._from_run_dto with exception (pywin32 300 (c:\python\lib\site-packages), Requirement.parse('pywin32==227; sys_platform == "win32"'), {'docker'}).


Azure ML SDK Version: 1.23.0


In [3]:
from azureml.core import Workspace


subscription_id = 'xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx'
resource_group = 'rg-sandbox'
workspace_name = 'sandboxaml'

ws = Workspace(subscription_id, resource_group, workspace_name)


In [4]:
print(ws.name, ws.location, ws.resource_group, ws.location, sep='\t')

sandboxaml	westeurope	rg-sandbox	westeurope


In [5]:
# Compute target creation

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


cpu_cluster_name = "compute-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Cluster already exists")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D3_V2',
                                                           min_nodes=0, max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count=0, timeout_in_minutes=30)

Creating...
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [6]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies


env = Environment("my_remote_env")

env.docker.enabled = True
env.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn',
                                                                          'pandas',
                                                                          'numpy',
                                                                          'joblib',                                    
                                                                          'matplotlib'
                                                                         ])
env.python.conda_dependencies.add_pip_package("inference-schema[numpy-support]")


In [7]:
experiment_name = 'my_remote_exp'

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)

exp

Name,Workspace,Report Page,Docs Page
my_remote_exp,sandboxaml,Link to Azure Machine Learning studio,Link to Documentation


In [8]:
%%writefile my_python_script.py

from azureml.core.run import Run


run = Run.get_context()
exp = run.experiment
ws = run.experiment.workspace

print(f"Hello wolrd, we are in the run : {run.id}")


Writing my_python_script.py


In [9]:
from azureml.core import ScriptRunConfig
from azureml.core.runconfig import DEFAULT_CPU_IMAGE


project_folder = '.'

# 1. Python file
src = ScriptRunConfig(source_directory=project_folder, script='my_python_script.py')

# 2. Set compute target to the one created in previous step
src.run_config.target = compute_target.name

# 3. Set python environment
src.run_config.environment = env

# 4. Some tags for the run
runtags= {"Type": "test" , 
          "Langage" : "Python" , 
          "Framework" : "Scikit-Learn", 
          "Team" : "MVP"}

In [10]:
run = exp.submit(config=src, tags=runtags)
run

Submitting C:\Users\Paul PETON\AppData\Local\Programs\Microsoft VS Code Insiders directory for run. The size of the directory >= 25 MB, so it can take a few minutes.


Experiment,Id,Type,Status,Details Page,Docs Page
my_remote_exp,my_remote_exp_1618088952_486cfc40,azureml.scriptrun,Preparing,Link to Azure Machine Learning studio,Link to Documentation


In [11]:
print("Status =", run.get_status())

Status = Preparing


In [12]:
run.get_details()

{'runId': 'my_remote_exp_1618088952_486cfc40',
 'target': 'compute-cluster',
 'status': 'Preparing',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '1ca55bf1-920c-42f2-8efb-e47c45b8b8ce'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'my_python_script.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'compute-cluster',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'priority': None,
  'credentialPassthrough': False,
  'identity': None,
  'environment': {'name': 'my_remote_env',
   'version': 'Autosave_2021-04-10T21:10:53Z_b8e44504',
   'python': {'interpreterPath': 'python',
    'userManagedDependencies': False,
    'condaDependencies': {'channels': ['anaconda', 'conda-forge'],
     'dependencies': ['python=3.6.2',
      